scrape instagram and facebook / reddit, yahoo answers, hestenettet for statements on pain

In [ ]:
pip install beautifulsoup4

TEXT GENERATION MODEL: https://huggingface.co/philschmid/flan-t5-base-samsum?text=hej

For this project, I am gonna use the Reddit API Praw. I am following this guide on how to implement it and parts of the code will be taken from here. https://towardsdatascience.com/scraping-reddit-data-1c0af3040768

In [ ]:
pip install psaw

In [ ]:
pip install praw

In [ ]:
# import necessary libraries 
import praw
from psaw import PushshiftAPI
import bs4
import requests
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from praw.models import MoreComments

In [ ]:
# authenticating myself!
reddit = praw.Reddit(client_id='Qbcp7XRs2X_HWLnshuyAAw',client_secret='yw9xGdrzGOn9wzE1qhVTZrPiAVa-5Q',user_agent='webscraper')

Now to the webscraping. 
I wanna make sure that the comments and subcomments I get with the keywords are actually about physical and mental pain, so I am limiting my search to a few subreddits that I chose manually using the search function on Reddit.

Part of the code is taken from:  https://praw.readthedocs.io/en/latest/tutorials/comments.html

In [ ]:
# list of key words
words_pain = ['pain','illnes','symptom','sick','treatment','diagnosed','undiagnosed','diagnosis','syndrome','drugs','drug','medication','meds','painkillers','disability','disease']

# list to hold scraped posts 
pain_all=[]

def get_comments():      
  for submission in reddit.subreddit('ChronicPain+ChronicIllness+backpain+disability+migraine+chronicpainrelief+kneepain+Fibromyalgia+painhub+painmed').hot(limit=None):          # iterating through subreddits
    submission.comments.replace_more(limit=None)          # iterate through all the comments 
    comment_queue = submission.comments[:]               # Seed with top-level
    id=submission.id
    title=str(submission.title)
    for word in words_pain:                              # iterate through keywords    
     if word in title or word in submission.comments:    # check if comment or title contains keywords 
      while comment_queue:
        comment = comment_queue.pop(0)
        #print(comment.body)
        #comment_queue.extend(comment.replies) 
        # get both titles and bodies 
        pain_all.append(title)
        pain_all.append(comment.body)
  return pain_all

get_comments()
print(len(pain_all))
print(pain_all)


In [ ]:
print('scraped content: ' ,pain_all)
print('amount of scraped content: ',len(pain_all))

# DATA WRANGLING
Now we have a list of 2764 posts scraped from 10 different subreddits. 

As I want to be able to use this dataset for a variety of things; for finetuning and as inputs to different AI models, I want it to be cleaned up and labelled, but I still want to keep the essence intact, that meaning all emojis and special characters aswell as it being case-sensitive. 

In [ ]:
import nltk 
from nltk.tokenize import sent_tokenize
import string
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# remove duplicates from list 
pain = [x for k,x in enumerate(pain_all) if x not in pain_all[:k]]

print('content with removed duplicates: ' ,pain)
print('number of comments: ',len(pain))

In [ ]:
# hold cleaned dataset 
pain_newlines = []

# remove new lines 
for item in pain:
  pain_newlines.append(item.replace('\n',''))    

# hold cleaned dataset 
pain_clean = []

for item in pain_newlines:
  if 'http' in item: 
    continue
  else: 
    pain_clean.append(item)  

In [ ]:
# label datasets 

pain_all=[]

# iterate over data to get text elements and index elements 
for index,text in enumerate(pain_clean):
  dictionary_all = {'text':text,'index':index}
  pain_all.append(dictionary_all)

# words that indicate a comment seeking advice
advice_seeking_words = ['do','how','?','which','what','who','when','whom','whose']

# hold the labeled data
data_labeled = []

# iterate over data 
for item in pain_all:
  text = item['text']
  tokens = nltk.word_tokenize(text)   
  pos_tags = nltk.pos_tag(tokens)

  if any([word in ['do','how','?','which','what','who','when','whom','whose'] for word,pos in pos_tags]):
   label = 1     # asking for advie
  else: 
    label = 0    # giving advice
  data_labeled.append({'text':text,'label':label})

len(data_labeled)


In [ ]:
# split into train and test 
pain_train = []
pain_test = []

pain_train.append(data_labeled[:1200])
pain_test.append(data_labeled[1200:])

# content is in 2d array - needs to be 1d 
pain_test = pain_test[0]
pain_train = pain_train[0]

print('test_len: ',len(pain_test))
print('train_len: ',len(pain_train))

In [ ]:
import json

# save data files
with open('data_pain_train.json','w',encoding='utf-8') as fout_train:   
  #fout.write(str(pain))
  json.dump(pain_train,fout_train)

with open('data_pain_test.json','w',encoding='utf-8') as fout_test: 
  json.dump(pain_test,fout_test)